In [1]:
import cv2
import numpy as np
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from ultralytics import YOLO
import math

In [2]:
# Charger le modèle YOLOv8
yolo = YOLO("./models/yolov8n.pt")

# Charger la cascade de Haar pour la détection de visage
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Charger le modèle de reconnaissance de célébrités
modelCNN = load_model("./models/celebrity_recognition.h5")

# Charger l'encodeur d'étiquettes
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('./models/label_encoder_classes.npy')

In [3]:
# Capture vidéo à partir d'un fichier
cap = cv2.VideoCapture('./input1.mp4')

# Obtention de la fréquence d'images par seconde (FPS) et des dimensions du cadre vidéo
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Configuration de l'encodeur vidéo pour la sortie
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

In [4]:
# Fonction pour prédire la classe d'une région d'intérêt (ROI) du visage
def predict_face_roi(face_roi, model, label_encoder):
    face_roi = cv2.resize(face_roi, (64, 64))  
    face_roi = np.array(face_roi) / 255.0
    face_roi = np.expand_dims(face_roi, axis=0)  

    predictions = modelCNN.predict(face_roi)
    predicted_class_index = np.argmax(predictions)
    predicted_class_accuracy = predictions[0, predicted_class_index]

    if predicted_class_accuracy < 0.96:
        return "inconnu", predicted_class_accuracy

    predicted_class_name = label_encoder.classes_[predicted_class_index]
    return predicted_class_name, predicted_class_accuracy

In [5]:
# Traiter chaque image (frame)
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Effectuer la détection d'objets
    results = yolo(frame, conf=0.6)

    # Traiter chaque personne détectée
    for result in results:
        boxes = result.boxes

        for box in boxes:
            cls = int(box.cls[0])

            if cls == 0:  # personne
                # boîte englobante
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convertir en valeurs entières

                # Dessiner une boîte englobante pour la personne
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                
                # Extraire la région d'intérêt (ROI)
                roi = frame[y1:y2, x1:x2]

                # Effectuer la détection de visage sur la ROI
                faces = face_cascade.detectMultiScale(roi, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

                # Sélectionner le meilleur visage en fonction de la surface
                selected_face = None
                max_area = 0

                for (fx, fy, fw, fh) in faces:
                    face_area = fw * fh
                    if face_area > max_area:
                        max_area = face_area
                        selected_face = (fx, fy, fw, fh)

                # Dessiner une boîte englobante pour le meilleur visage
                if selected_face is not None:
                    (fx, fy, fw, fh) = selected_face
                    cv2.rectangle(frame, (x1 + fx, y1 + fy), (x1 + fx + fw, y1 + fy + fh), (255, 0, 0), 2)

                    # Extraction de la région d'intérêt du visage
                    face_roi = cv2.resize(frame[y1 + selected_face[1]:y1 + selected_face[1] + selected_face[3],
                                        x1 + selected_face[0]:x1 + selected_face[0] + selected_face[2]], (256, 256))
                    face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)

                    # Prédiction de la célébrité dans la région d'intérêt du visage
                    result, confidence = predict_face_roi(frame[y1 + selected_face[1]:y1 + selected_face[1] + selected_face[3],
                                                    x1 + selected_face[0]:x1 + selected_face[0] + selected_face[2]], modelCNN, label_encoder)

                    # Affichage du résultat de la prédiction
                    label = f"{result} ({confidence:.2%} Confiance)" if result != "inconnu" else "Inconnu"

                    # Afficher le texte "face" à l'intérieur de la boîte du visage
                    cv2.putText(frame, label, (x1 + fx -40, y1 + fy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Écrire le cadre traité dans la vidéo de sortie
    video_out.write(frame)
    # Afficher le cadre avec les détections
    cv2.imshow("Détection de personne YOLOv8", frame)

    # Quitter lorsque 'q' est pressé
    if cv2.waitKey(1) == ord('q'):
        break

# Libérer les ressources
cap.release()
video_out.release()
cv2.destroyAllWindows()


0: 384x640 1 person, 187.3ms
Speed: 0.0ms preprocess, 187.3ms inference, 12.8ms postprocess per image at shape (1, 3, 384, 640)
1/1 [==============================] - 0s 267ms/step

0: 384x640 1 person, 195.4ms
Speed: 0.0ms preprocess, 195.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 [==============================] - 0s 38ms/step

0: 384x640 1 person, 170.8ms
Speed: 0.0ms preprocess, 170.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 [==============================] - 0s 31ms/step

0: 384x640 1 person, 154.2ms
Speed: 19.1ms preprocess, 154.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 [==============================] - 0s 47ms/step

0: 384x640 1 person, 160.9ms
Speed: 4.1ms preprocess, 160.9ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)
1/1 [==============================] - 0s 23ms/step

0: 384x640 1 person, 209.7ms
Speed: 15.6ms preprocess, 209.7ms inference, 4.2ms postprocess per 